<a href="https://colab.research.google.com/github/PedroTonus/praticasGSI073/blob/main/Aula02_tokenizacao_pratica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aula 2 - Tokenização

In [ ]:
sentences = [
"O gato observou o pássaro na janela",
"O menino chutou a bola no quintal",
"A chuva molhou a rua inteira",
"O vento balançou as árvores altas",
"A mãe preparou o café da manhã",
"O cachorro encontrou um graveto grande",
"A professora explicou a tarefa difícil",
"O piloto guiou o avião suavemente",
"A criança abraçou o ursinho favorito",
"O artista pintou um quadro colorido",
"A lua iluminou o campo escuro",
"O músico afinou o violão antigo",
"A menina leu um livro interessante",
"O carteiro entregou uma carta urgente",
"A lâmpada clareou o quarto frio",
"O surfista enfrentou uma onda gigante",
"A bailarina praticou um salto perfeito",
"O escritor iniciou um novo capítulo",
"A estrela brilhou no céu limpo",
"O trem atravessou o túnel longo",
"A borboleta pousou na flor vermelha",
"O gato bebeu água fresca",
"A médica examinou o paciente nervoso",
"O turista tirou fotos da montanha",
"A planta cresceu rapidamente na varanda",
"O menino encontrou uma moeda dourada",
"A menina desenhou um castelo bonito",
"O pescador lançou a rede no lago",
"A neve cobriu a estrada vazia",
"O garoto abriu um pacote misterioso",
"A cantora ensaiou uma melodia suave",
"O marceneiro cortou a madeira grossa",
"A vela iluminou o corredor sombrio",
"O bebê segurou o dedo da mãe",
"A bicicleta atropelou uma poça de água",
"O carteiro encontrou o portão fechado",
"A sopa esquentou a cozinha fria",
"O trem apitou na estação central",
"A árvore abrigou vários pássaros",
"O aluno terminou o exercício rápido",
"A nuvem bloqueou o sol forte",
"O gato arranhou o sofá novo",
"A girafa alcançou as folhas altas",
"O menino soprou uma bolha enorme",
"A mulher guardou as roupas limpas",
"O relógio marcou meia-noite",
"O leão rugiu para o horizonte",
"A menina escondeu um bilhete secreto",
"O pintor limpou seus pincéis velhos",
"A tartaruga atravessou o gramado",
"O avião sobrevoou a cidade grande",
"A atriz decorou um texto difícil",
"O cachorro cavou um buraco fundo",
"A porta rangiu com o vento",
"O garoto segurou o chapéu vermelho",
"A avó preparou um bolo cheiroso",
"O gato pulou sobre a cadeira",
"A abelha pousou na pétala rosa",
"O menino dobrou o papel cuidadosamente",
"A música encheu o salão vazio",
"O sol aqueceu as pedras da praia",
"A menina encontrou um colar brilhante",
"O homem fechou a janela depressa",
"A moto passou pela rua estreita",
"O cavalo correu pelo campo verde",
"A criança segurou um balão azul",
"O jardineiro cortou a grama alta",
"O motorista virou na esquina errada",
"A bailarina amarrou as sapatilhas",
"O peixe nadou entre as algas",
"A garota escreveu uma carta longa",
"O atleta correu para a linha final",
"A porta bateu com força",
"O professor recolheu os trabalhos",
"O gato derrubou um vaso pequeno",
"A gaivota sobrevoou a praia cheia",
"O garoto perdeu o lápis preto",
"A menina cuidou da planta frágil",
"O carro freou na faixa de pedestres",
"A chuva refrescou a tarde quente",
"O ator improvisou uma fala engraçada",
"A bandeira balançou ao vento",
"O cachorro encontrou um osso velho",
"A mãe organizou os brinquedos espalhados",
"O violinista tocou uma nota longa",
"A estrela guiou os viajantes cansados",
"O detetive analisou uma pista nova",
"A criança escondeu o doce favorito",
"O vento moveu a cortina leve",
"A maré trouxe conchas coloridas",
"O telefone tocou inesperadamente",
"A menina abraçou a amiga triste",
"O gato perseguiu um inseto pequeno",
"A porta abriu com dificuldade",
"O menino guardou o carrinho azul",
"A cozinheira temperou a carne fresca",
"O repórter escreveu uma matéria rápida",
"A cabra atravessou a ponte velha",
"O jogador marcou um ponto importante",
"A bailarina sorriu para o público",
"O cantineiro serviu a refeição quente",
"A água escorreu pela calha metálica",
"O pássaro cantou no galho fino",
"A enfermeira aplicou uma injeção leve",
"O garoto montou um quebra-cabeça difícil",
"A sombra cobriu parte do chão",
"O trem diminuiu a velocidade",
"A menina segurou o guarda-chuva azul",
"O gato explorou a casa inteira",
"A mãe lavou a louça rapidamente",
"O cachorro latiu para a porta",
"A criança pulou sobre a poça",
"O livro caiu da estante alta",
"O avião pousou com suavidade",
"A estrela piscou levemente no céu",
"O gato dormiu no tapete macio",
"A menina penteou os cabelos longos",
"O atleta saltou sobre o obstáculo",
"A abelha voou até outra flor",
"O menino jogou a bola longe",
"A brisa refrescou a manhã clara",
"O sapo pulou no lago raso",
"A enfermeira sorriu para o paciente",
"O pintor escolheu uma cor vibrante",
"A menina cantou uma canção doce",
"O gato se escondeu atrás da porta",
"A chuva caiu sobre o telhado velho",
"O pescador recolheu a rede pesada",
"A moça arrumou a mesa do jantar",
"O cão encontrou um brinquedo velho",
"A estrela iluminou o caminho curto",
"O motorista estacionou o carro preto",
"A criança desenhou um sol amarelo",
"O vento virou a página do livro",
"A garota colocou flores no vaso",
"O homem apagou a luz da sala",
"A bailarina girou com leveza",
"O gato observou as pessoas na rua",
"A menina abriu um presente rosa",
"O barco balançou nas ondas calmas",
"O menino correu para o portão",
"A professora escreveu no quadro branco",
"O estudante revisou o texto final",
"A maré subiu lentamente",
"O músico tocou um acorde suave",
"A criança segurou um brinquedo novo",
"O carro acelerou na avenida larga",
"A nuvem trouxe um pouco de sombra",
"O gato saltou sobre a mesa",
"A menina coloriu a folha inteira",
"O homem empurrou a porta pesada",
"A mulher acendeu uma vela pequena",
"O pescador encontrou um peixe grande",
"A menina soprou as velas do bolo",
"O relógio despertou bem cedo",
"A estrela caiu no horizonte",
"O cão buscou a bolinha verde",
"A criança montou uma torre alta",
"O menino derrubou a caixa vazia",
"A mãe secou os pratos limpos",
"O gato arranhou a porta de madeira",
"A menina escreveu no caderno novo",
"O músico ajustou o microfone alto",
"A árvore balançou com a ventania",
"O amigo segurou a mochila pesada",
"A criança observou o céu azul",
"O homem procurou as chaves perdidas",
"A borboleta voou sobre o jardim",
"O cachorro abanou o rabo feliz",
"A garota prendeu o cabelo curto",
"O menino encontrou um chapéu velho",
"A lâmpada iluminou o corredor longo",
"O peixe saltou fora d'água",
"A menina segurou a caixa colorida",
"O carro desviou do buraco fundo",
"A mãe abraçou o filho pequeno",
"O gato bebeu leite morno",
"A nuvem se moveu rapidamente",
"O atleta praticou um novo movimento",
"A criança abriu a porta devagar",
"O músico tocou o piano antigo",
"A borboleta descansou na pedra",
"O vento espalhou as folhas secas",
"A menina procurou o brinquedo perdido",
"O homem iniciou uma conversa curta",
"A água escorreu pela parede fria",
"O gato caminhou pelo corredor",
"A moça encontrou uma carta velha",
"O menino riscou o papel branco",
"A menina segurou o ursinho marrom",
"O carro virou à direita rapidamente",
"A abelha pousou na mesa",
"O cachorro roeu o osso novo",
"A estrela brilhou sobre o mar",
"O menino vestiu a jaqueta azul",
"A criança correu no parquinho",
"O gato caiu do sofá macio",
"A mãe preparou a mochila escolar",
"O estudante apagou o quadro sujo",
"A árvore cresceu na praça central",
"O vento levantou poeira fina",
"A menina tocou a campainha",
"O homem abriu uma gaveta pesada",
"A água invadiu o corredor",
"O cachorro lambeu o prato vazio",
"O gato pulou para o colo da dona",
"A brisa moveu o cabelo da menina",
"O menino colocou o boné vermelho",
"A criança chutou a bola amarela",
"O carro entrou na garagem pequena",
"A professora fechou o livro grosso",
"O artista expôs um quadro novo",
"A menina encontrou um anel brilhante",
"O pássaro pousou no muro baixo",
"A chuva apagou as marcas do chão",
"O gato ronronou para a dona",
"A vela derreteu lentamente",
"O menino escondeu a caixa pequena",
"A mãe ajeitou o lençol da cama",
"O avião sobrevoou o oceano azul",
"A nuvem cobriu a montanha branca",
"O gato arrastou o pano rasgado",
"A criança abraçou o pai sorrindo",
"O vento derrubou uma garrafa vazia",
"A menina fechou o caderno rosa",
"O cachorro entrou na casinha quente",
"A luz refletiu no espelho limpo",
"O homem encontrou uma carteira",
"A criança soprou bolhas de sabão",
"A moça escreveu uma mensagem rápida"
]

file_path = "corpus.txt"

with open(file_path, "w", encoding="utf-8") as f:
    for sentence in sentences:
        f.write(sentence + "\n")

print(f"File '{file_path}' created successfully with {len(sentences)} sentences.")

# Optionally, display the content to verify
# with open(file_path, "r", encoding="utf-8") as f:
#     content = f.read()
#     print("\nContent of corpus.txt:\n", content)

## Parte 1 - Pré-tokenização


In [ ]:
from tokenizers import Tokenizer, models, pre_tokenizers

tok_ws = Tokenizer(models.BPE())
tok_ws.pre_tokenizer = pre_tokenizers.Whitespace()
frase = "O menino voou no céu azul"

print(tok_ws.pre_tokenizer.pre_tokenize_str(frase))


## *Punctuation + Whitespace*

In [ ]:
tok_punc = Tokenizer(models.BPE())
tok_punc.pre_tokenizer = pre_tokenizers.Sequence([
    pre_tokenizers.Whitespace(),
    pre_tokenizers.Punctuation()
])
print(tok_punc.pre_tokenizer.pre_tokenize_str(frase))

**Pretokenizer: ByteLevel - estilo GPT-2**

In [ ]:
tok_byte = Tokenizer(models.BPE())
tok_byte.pre_tokenizer = pre_tokenizers.ByteLevel()
print(tok_byte.pre_tokenizer.pre_tokenize_str(frase))

# Metaspace (SentencePiece style)

In [ ]:
tok_meta = Tokenizer(models.BPE())
tok_meta.pre_tokenizer = pre_tokenizers.Metaspace()
print(tok_meta.pre_tokenizer.pre_tokenize_str(frase))

#Treinamento


In [ ]:
# 02_tokenizer_train.ipynb

from tokenizers import Tokenizer, models, trainers, pre_tokenizers

# 1. Visão geral do algoritmo BPE
print("Treinar o tokenizador (BPE):\n")
print("1. Comece com todos os caracteres presentes no corpus como tokens.")
print("2. Encontre e una o par de tokens mais frequente em um novo token.")
print("3. Repita até atingir o tamanho de vocabulário desejado.\n")

# 2. Corpus de treino
file_path = "corpus.txt"
with open(file_path, "r", encoding="utf-8") as f:
    corpus = [line.strip() for line in f if line.strip()]
print("Corpus de treino (primeiras 230 frases):", corpus[:230], "\n")

# 3. Configuração do tokenizador
tokenizer = Tokenizer(models.BPE(unk_token="<unk>"))
tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()

bpe_trainer = trainers.BpeTrainer(
    vocab_size=100,
    special_tokens=["<pad>", "<unk>", "<s>", "</s>"]
)

# 4. Treinamento
tokenizer.train_from_iterator(corpus, trainer=bpe_trainer)
vocab = tokenizer.get_vocab()
print(f"Tamanho do vocabulário: {len(vocab)}\n")

# 5. Visualizando parte do vocabulário
sorted_vocab = sorted(vocab.items(), key=lambda kv: kv[1])[:20]
for token, idx in sorted_vocab:
    print(f"{idx:>3} \u2192 {repr(token)}")

# 6. Salvando e recarregando
tokenizer.save("bpe_tokenizer.json")
tokenizer_new = Tokenizer.from_file("bpe_tokenizer.json")

# 7. Testando em novas frases
textos = ["O rato roeu a roupa do rei de Roma", "Hello, world."]

print("\nTokenização de exemplos:")
for texto in textos:
    out = tokenizer_new.encode(texto)
    print(f"Texto: {texto}")
    print(f"Tokens: {out.tokens}")
    print(f"IDs: {out.ids}\n")

# Encode

In [ ]:
# 03_tokenizer_encode.ipynb
# Pipeline de tokenização: normalização → pré-tokenização → modelo → pós-processamento

from tokenizers import Tokenizer, normalizers, pre_tokenizers, processors
from tokenizers.normalizers import NFD, StripAccents, Lowercase
from tokenizers.pre_tokenizers import Whitespace, Digits, Sequence
from tokenizers.processors import TemplateProcessing

print("### Pipeline de tokenização ###")
print(" Normalization")
print(" Pre-tokenization")
print(" Model")
print(" Post-processing\n")

# Carregar o tokenizador treinado (BPE)
tokenizer = Tokenizer.from_file("bpe_tokenizer.json")

# -----------------------------------------------------------
# Normalization
# -----------------------------------------------------------
print("# Normalization")
normalizer = normalizers.Sequence([
    NFD(),          # decomposição de acentos
    Lowercase(),    # tudo minúsculo
    StripAccents()  # remove acentos
])
texto = "Héllò hôw are ü?"
print("Antes:", texto)
print("Depois:", normalizer.normalize_str(texto), "\n")
tokenizer.normalizer = normalizer

# -----------------------------------------------------------
# Pre-tokenization
# -----------------------------------------------------------
print("# Pre-tokenization")
pre_tok = Sequence([
    Whitespace(),
    Digits(individual_digits=True)
])
texto2 = "Hello! How are you? Tenho R$ 213,12."
print("Pré-tokenização:", pre_tok.pre_tokenize_str(texto2), "\n")
tokenizer.pre_tokenizer = pre_tok

# -----------------------------------------------------------
# Model
# -----------------------------------------------------------
print("# Model: BPE (Byte Pair Encoding)")
# já carregado do arquivo bpe_tokenizer.json

# -----------------------------------------------------------
# Post-processing
# -----------------------------------------------------------
print("# Post-processing (TemplateProcessing)")
tokenizer.post_processor = TemplateProcessing(
    single="[CLS] $A [SEP]",
    pair="[CLS] $A [SEP] $B:1 [SEP]:1",
    special_tokens=[("[CLS]", 1), ("[SEP]", 2)],
)

# -----------------------------------------------------------
# Aplicando tudo
# -----------------------------------------------------------
encoded = tokenizer.encode("olá mundo")
print("Tokens IDs:", encoded.ids)
print("Tokens:", encoded.tokens)


## Bytelevel vs SentencePiece

In [ ]:
# 03_bytelevel_vs_sentencepiece.ipynb
# Comparando ByteLevel (GPT-2) vs SentencePiece (mT5)

from transformers import AutoTokenizer
import unicodedata

# -----------------------------
# 1️⃣ Modelos
# -----------------------------
BYTELEVEL_MODEL = "openai-community/gpt2"
SENTPIECE_MODEL = "google/mt5-small"

tok_byte = AutoTokenizer.from_pretrained(BYTELEVEL_MODEL)
tok_spm  = AutoTokenizer.from_pretrained(SENTPIECE_MODEL)

# Garantir pad_token
if tok_byte.pad_token is None and hasattr(tok_byte, "eos_token"):
    tok_byte.pad_token = tok_byte.eos_token

# -----------------------------
# 2️⃣ Texto de exemplo
# -----------------------------
text = "Vamos comer, vovó! 🙂"
print(f"Texto: {text}\n")

# -----------------------------
# 3️⃣ Tokenização
# -----------------------------
def encode_details(tokenizer, name):
    enc = tokenizer(text, add_special_tokens=True, return_offsets_mapping=True)
    tokens = tokenizer.convert_ids_to_tokens(enc["input_ids"])
    ids = enc["input_ids"]
    offsets = enc["offset_mapping"]
    print(f"=== {name} ===")
    print("Tokens:", tokens)
    print("IDs:", ids)
    print("Qtd tokens:", len(tokens))
    print("Decoded:", tokenizer.decode(ids))
    print("Offsets:", offsets)
    print()

encode_details(tok_byte, "ByteLevel (GPT-2)")
encode_details(tok_spm, "SentencePiece (mT5)")

# -----------------------------
# 4️⃣ Comparação Unicode (opcional)
# -----------------------------
def show_unicode_chars(s):
    for ch in s:
        name = unicodedata.name(ch, "UNKNOWN")
        print(f"{repr(ch)} -> {name}")

print("\nCaracteres Unicode do texto:")
show_unicode_chars(text)


# Avaliação

In [ ]:
from tokenizers import Tokenizer
import numpy as np

# Carrega o tokenizador treinado
tokenizer = Tokenizer.from_file("bpe_tokenizer.json")

# Corpus de teste (pode ser parte do seu corpus real)
test_texts = [
    "O rato roeu a roupa do rei de Roma.",
    "Aprender tokenização é divertido!",
    "GPT-2 e mT5 usam abordagens diferentes.",
    "Python é ótimo para NLP 😄",
]

# Funções auxiliares
def count_chars(text):
    return len(text)

def count_words(text):
    return len(text.split())

def evaluate_tokenizer(tokenizer, texts):
    stats = []
    for t in texts:
        enc = tokenizer.encode(t)
        stats.append({
            "text": t,
            "chars": count_chars(t),
            "words": count_words(t),
            "tokens": len(enc.tokens),
            "unk": enc.tokens.count("<unk>"),
            "decoded_ok": (tokenizer.decode(enc.ids) == t)
        })
    return stats

stats = evaluate_tokenizer(tokenizer, test_texts)

# Converter para métricas agregadas
import pandas as pd
df = pd.DataFrame(stats)

tpc = (df["tokens"] / df["chars"]).mean()
tpw = (df["tokens"] / df["words"]).mean()
unk_rate = (df["unk"].sum() / df["tokens"].sum()) * 100
decode_acc = (df["decoded_ok"].mean()) * 100

print("=== Métricas de eficiência ===")
print(f"Tokens por caractere (TPC): {tpc:.3f}")
print(f"Tokens por palavra (TPW): {tpw:.3f}")
print(f"Percentual de <unk>: {unk_rate:.2f}%")
print(f"Reversibilidade (decode == original): {decode_acc:.1f}%")
print(f"Tamanho médio da sequência: {df['tokens'].mean():.1f} tokens/frase")
